In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    dict(
        user_id=np.random.randint(0, 1000, size=100000),
        n_imp=np.random.randint(1, 101, size=100000),
    )
)
df["n_click"] = df.n_imp.apply(lambda x: np.random.randint(0, int(np.ceil((x+1)*.3))))
df

,user_id,n_imp,n_click
0,465,90,4
1,330,44,1
2,930,51,9
3,705,56,4
4,812,90,17
...,...,...,...
99995,180,18,5
99996,967,96,2
99997,630,89,25
99998,384,89,16


In [3]:
from typing import Generator
from sklearn.model_selection import train_test_split


def split_data(user_ids: np.ndarray, n: int, random_state: int = 42) -> Generator[np.ndarray, None, None]:
    for i in range(n-1):
        uids, user_ids = train_test_split(user_ids, train_size=1/(n-i), random_state=random_state)
        yield uids
    yield user_ids


def data_loader(n: int, random_state: int) -> list[pd.DataFrame]:
    return [
        df[df.user_id.isin(uids)]
        for uids in split_data(df.user_id.unique(), n, random_state)
    ]

In [4]:
from itertools import combinations

from t_ab.ctr import ImpressionBasedCTRTtest, UserBasedCTRTtest

_ibtest = ImpressionBasedCTRTtest("user_id", "n_imp")
_ubtest = UserBasedCTRTtest("user_id", "n_imp")


def ibtest(dfs: list[pd.DataFrame]) -> float:
    return [
        _ibtest(dfa, dfb, "n_click")[1].pvalue
        for dfa, dfb in combinations(dfs, 2)
    ]


def ubtest(dfs: list[pd.DataFrame]) -> float:
    return [
        _ubtest(dfa, dfb, "n_click")[1].pvalue
        for dfa, dfb in combinations(dfs, 2)
    ]

In [5]:
from functools import partial
from t_ab.aa import AATest

aa_test = AATest(partial(data_loader, 3))

In [6]:
results = aa_test(ibtest, ubtest)

In [7]:
for result in results:
    print(result.multipletests_result, result.is_rejected)

(array([False, False, False]), array([0.71555974, 0.12628875, 0.12628875]), 0.016952427508441503, 0.016666666666666666) False
(array([False, False, False]), array([0.71282197, 0.18133179, 0.12888656]), 0.016952427508441503, 0.016666666666666666) False
